In [9]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint

from keras.preprocessing import image

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,GlobalAveragePooling2D,Dropout
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [37]:
model_resnet50 = ResNet50(weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 1s 0us/step


In [38]:
for layer in model_resnet50.layers:
    layer.trainable = False

In [4]:
img_width, img_height = 150, 150
train_data_dir = '/content/drive/MyDrive/train_new'
val_data_dir = '/content/drive/MyDrive/test_new'
model_weights_file = 'model_resnet50.h5'
nb_train_samples = 4
nb_val_samples = 4
nb_epochs = 5

In [5]:
put1 = Input(shape=(img_width, img_height, 3))
output_resnet = model_resnet50(put1)
x = Flatten()(output_resnet)
x = Dense(64, activation='relu')(x)
x = Dense(10, activation='softmax')(x)
model=Model(inputs=put1,outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                        batch_size=32,class_mode='categorical')

Found 7414 images belonging to 10 classes.
Found 1847 images belonging to 10 classes.


**ACCURACY**

In [7]:
history = model.fit_generator( train_generator,  epochs=4,
  steps_per_epoch=len(train_generator),
  validation_steps=len(validation_generator))

Epoch 1/4
232/232 [==============================] - 1791s 8s/step - loss: 2.0246 - accuracy: 0.3427
Epoch 2/4
232/232 [==============================] - 1719s 7s/step - loss: 0.9460 - accuracy: 0.6917
Epoch 3/4
232/232 [==============================] - 1697s 7s/step - loss: 0.6284 - accuracy: 0.8036
Epoch 4/4
232/232 [==============================] - 1688s 7s/step - loss: 0.4698 - accuracy: 0.8551


**CONFUSION MATRIX AND F1_SCORE**

In [10]:
batch_size = 32
target_names = ['beagle', 'chihuahua', 'doberman', 'french_bulldog', 'golden_retriever', 'malamute', 'pug', 'saint_bernard', 'scottish_deerhound', 'tibetan_mastiff']
Y_pred = model.predict_generator(validation_generator, 2513 // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Classification Report')
print(classification_report(validation_generator.classes, y_pred))

Classification Report
              precision    recall  f1-score   support

           0       0.17      0.08      0.11       231
           1       0.12      0.12      0.12       143
           2       0.08      0.08      0.08       165
           3       0.09      0.05      0.07       154
           4       0.07      0.06      0.06       153
           5       0.08      0.08      0.08       187
           6       0.10      0.21      0.14       209
           7       0.08      0.07      0.08       187
           8       0.17      0.16      0.16       275
           9       0.11      0.11      0.11       143

    accuracy                           0.11      1847
   macro avg       0.11      0.10      0.10      1847
weighted avg       0.11      0.11      0.11      1847



**roc_curve**

In [35]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
rc=roc_curve(validation_generator.classes, y_pred, pos_label=2)
print(rc)

(array([0.        , 0.07669441, 0.21819263, 0.31807372, 0.55469679,
       0.65576694, 0.72532699, 0.77348395, 0.86741974, 0.93935791,
       1.        ]), array([0.        , 0.12121212, 0.26666667, 0.35151515, 0.53333333,
       0.63030303, 0.71515152, 0.76969697, 0.84848485, 0.93939394,
       1.        ]), array([10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0]))


**confusion matrix**

In [12]:
print('Confusion Matrix')
cm = confusion_matrix(validation_generator.classes, y_pred)
print(cm)

Confusion Matrix
[[19  5 24 11 18 27 56 26 29 16]
 [ 9 17 13  2  8 13 25 18 24 14]
 [10 15 13  9 14 16 30 14 24 20]
 [11 10  8  8 13 19 42 11 21 11]
 [ 6 15 16  9  9 15 35 18 23  7]
 [10 14 18  8 13 15 56 20 18 15]
 [12 16 22 11 22 17 44 17 35 13]
 [12 14 26 11  8 18 46 14 21 17]
 [15 21 20 12 18 33 60 32 44 20]
 [ 8  9 11  9  8 13 34 12 23 16]]


In [13]:
import os

In [23]:
class_name=[a for a in os.listdir('/content/drive/MyDrive/train_new')]
class_names=sorted(class_name)
print(class_names)

['beagle', 'chihuahua', 'doberman', 'french_bulldog', 'golden_retriever', 'malamute', 'pug', 'saint_bernard', 'scottish_deerhound', 'tibetan_mastiff']


In [16]:
pip install gradio

     |████████████████████████████████| 1.6MB 5.6MB/s 
     |████████████████████████████████| 1.9MB 28.4MB/s 
     |████████████████████████████████| 215kB 34.7MB/s 
     |████████████████████████████████| 962kB 26.9MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 3.2MB 31.2MB/s 
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-cp37-none-any.whl size=3372 sha256=037f6dba457b31c651d5db0f9ccb7e816686936ad450958a58dbdf175fbea514
  Stored in directory: /root/.cache/pip/wheels/9f/fc/a7/ab5712c3ace9a8f97276465cc2937316ab8063c1fea488ea77
  Created wheel for ffmpy: filename=ffmpy-0.3.0-cp37-none-any.whl size=4710 sha256=81a38c4090763fd1d4c87f63463449662c09fc13caaf5687c9d83602c1a72b93
  Stored in directory: /root/.cache/pip/wheels/cc/ac/c4/bef572cb7e52bfca170046f567e64858632daf77e0f34e5a74
Successfully built flask-cachebuster ffmpy


In [26]:
import gradio as gr

In [27]:
def predict_image(img):
  img_4d=img.reshape(-1,150,150,3)
  prediction=model.predict(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(10)}

In [ ]:
image = gr.inputs.Image(shape=(150,150))
label = gr.outputs.Label(num_top_classes=10)
r=gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')
r.launch()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://48323.gradio.app
Interface loading below...
